# Build the logbook for Hologram with empty or filters

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNES, DESC-LSST fullmember, behalf AuxTel Teal VERA C. Rubin observatory
- creation date : December 8th 2022
- last update : December 8th 2022


Works directory per directory (day by day)

CCIN2P3:

- works with kernels **anaconda3_auxtel** (with libradtran) at CC
- works with kernel **python 3** locally

In [1]:
import sys
sys.path

['/pbs/throng/lsst/users/dagoret/desc/AuxTelComm/notebookccdm_gen3/runspectractor_standalone/2022_12',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3_nov2022/lib/python39.zip',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3_nov2022/lib/python3.9',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3_nov2022/lib/python3.9/lib-dynload',
 '',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3_nov2022/lib/python3.9/site-packages',
 '__editable__.rubin_sim-0.13.1.finder.__path_hook__',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3_nov2022/lib/python3.9/site-packages/Spectractor-2.3-py3.9.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3_nov2022/lib/python3.9/site-packages/libradtranpy-0.1.0-py3.9.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3_nov2022/lib/python3.9/site-packages/Django-4.1.3-py3.9.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3_nov2022/lib/python3.9/site-packages/sqlparse-0.4.3-py3.9.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3_nov2022/lib/python3.9/sit

In [2]:
! ls /sps/lsst/groups/auxtel/data/2023/empty~holo4_003/20221207

exposure_2022120700357_postisrccd.fits	exposure_2022120700412_postisrccd.fits
exposure_2022120700361_postisrccd.fits	exposure_2022120700416_postisrccd.fits
exposure_2022120700362_postisrccd.fits	exposure_2022120700417_postisrccd.fits
exposure_2022120700366_postisrccd.fits	exposure_2022120700421_postisrccd.fits
exposure_2022120700367_postisrccd.fits	exposure_2022120700422_postisrccd.fits
exposure_2022120700371_postisrccd.fits	exposure_2022120700426_postisrccd.fits
exposure_2022120700372_postisrccd.fits	exposure_2022120700427_postisrccd.fits
exposure_2022120700376_postisrccd.fits	exposure_2022120700431_postisrccd.fits
exposure_2022120700377_postisrccd.fits	exposure_2022120700432_postisrccd.fits
exposure_2022120700381_postisrccd.fits	exposure_2022120700436_postisrccd.fits
exposure_2022120700382_postisrccd.fits	exposure_2022120700437_postisrccd.fits
exposure_2022120700386_postisrccd.fits	exposure_2022120700442_postisrccd.fits
exposure_2022120700387_postisrccd.fits	exposure_2022120700443_po

# Select the combination DATE, FILTER, DISPERSER

In [3]:
DATE = '20221207'
filterdispersernames = ["empty~holo4_003","BG40~holo4_003","FELH0600~holo4_003","SDSSg~holo4_003",\
                        "empty~ronchi170lpmm","BG40~ronchi170lpmm","FELH0600~ronchi170lpmm","SDSSg~ronchi170lpmm",\
                       ]
filterdispersername = filterdispersernames[0]
version="v6.0"  # new orientation

# If position need to be specified add it in this dictionnary

In [4]:
# reconstructed / good /xpos /ypos

if version == "v6.0":
    info={\

          # with red filter
          'exposure_2022031600351_postisrccd.fits':(1,1,300.,1700),\
         }
        

# Imports

In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re

plt.rcParams["figure.figsize"] = (16,4)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [6]:
from astropy.io import fits

In [7]:
import imp
MODULE_EXTENSIONS = ('.py', '.pyc', '.pyo')

/scratch/users/d/dagoret/ipykernel_22011/3547059130.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [8]:
def package_contents(package_name):
    file, pathname, description = imp.find_module(package_name)
    if file:
        raise ImportError('Not a package: %r', package_name)
    # Use a set because some may be both source and compiled.
    return set([os.path.splitext(module)[0]
        for module in os.listdir(pathname)
        if module.endswith(MODULE_EXTENSIONS)])

In [9]:
package_contents("spectractor")

{'__init__',
 '_version',
 'astrometry',
 'config',
 'logbook',
 'parameters',
 'tools'}

In [10]:
package_contents("spectractor/extractor")

{'__init__',
 'background',
 'chromaticpsf',
 'dispersers',
 'extractor',
 'images',
 'psf',
 'spectroscopy',
 'spectrum',
 'targets'}

# Initialisation

In [11]:
dir_logbooks="logbooks"

In [12]:
import os
if not os.path.exists(dir_logbooks):
    os.makedirs(dir_logbooks)

## Get inputs files and sort them according date and number

In [13]:
home=os.getenv("HOME")

In [14]:
image_path=filterdispersername
logbook_csv=os.path.join(dir_logbooks,"auxtellogbook_" + filterdispersername + "_"+ DATE +"_" + version + ".csv")
logbook_xlsx=os.path.join(dir_logbooks,"auxtellogbook_" + filterdispersername + "_" +DATE +"_" + version + ".xlsx")

# Path

In [15]:
HOSTCC=True
LAPTOP=False

In [16]:
# Set path depending on which computer running (according HOSTCC)
if HOSTCC:
    path_auxtel="/sps/lsst/groups/auxtel"
    path_spectractor=os.path.join(path_auxtel,"softs/github/desc/Spectractor")
    path_spectractor_config=os.path.join(path_spectractor,"config")
    path_images=os.path.join(path_auxtel,"data/2023/"+filterdispersername+"/"+DATE)

else:
    if not LAPTOP:
        path_auxtel="/Users/sylvie/DATA/AuxTelData2022"
        path_spectractor=os.path.join(path_auxtel,"/Users/sylvie/MacOSX/GitHub/LSST/Spectractor")
        path_spectractor_config=os.path.join(path_spectractor,"config")
        path_images=os.path.join(path_auxtel,"data/2022/"+filterdispersername+"/"+DATE)
        
    else:
        path_auxtel="/Users/dagoret/DATA/AuxTelData2022"
        path_spectractor=os.path.join(path_auxtel,"/Users/dagoret/MacOSX/GitHub/LSST/Spectractor")
        path_spectractor_config=os.path.join(path_spectractor,"config")
        path_images=os.path.join(path_auxtel,"data/2023/"+filterdispersername+"/"+DATE)
        

# Check the images exists

In [17]:
dir_images=path_images

In [18]:
all_files=os.listdir(dir_images)

In [19]:
all_files

['exposure_2022120700371_postisrccd.fits',
 'exposure_2022120700436_postisrccd.fits',
 'exposure_2022120700431_postisrccd.fits',
 'exposure_2022120700417_postisrccd.fits',
 'exposure_2022120700421_postisrccd.fits',
 'exposure_2022120700426_postisrccd.fits',
 'exposure_2022120700362_postisrccd.fits',
 'exposure_2022120700459_postisrccd.fits',
 'exposure_2022120700464_postisrccd.fits',
 'exposure_2022120700402_postisrccd.fits',
 'exposure_2022120700361_postisrccd.fits',
 'exposure_2022120700376_postisrccd.fits',
 'exposure_2022120700386_postisrccd.fits',
 'exposure_2022120700454_postisrccd.fits',
 'exposure_2022120700442_postisrccd.fits',
 'exposure_2022120700367_postisrccd.fits',
 'exposure_2022120700387_postisrccd.fits',
 'exposure_2022120700381_postisrccd.fits',
 'exposure_2022120700391_postisrccd.fits',
 'exposure_2022120700377_postisrccd.fits',
 'exposure_2022120700412_postisrccd.fits',
 'exposure_2022120700401_postisrccd.fits',
 'exposure_2022120700463_postisrccd.fits',
 'exposure_

In [20]:
all_selected_files = []
for filename in all_files:
    if re.search(".fits$",filename):
        all_selected_files.append(filename)
N=len(all_selected_files)

In [21]:
all_selected_files=np.array(all_selected_files)

### Sort files

In [22]:
def file_tag_forsorting(filename):
    #m=re.findall('^Cor_holo4_003_.*([0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]_.*)[.]fits$',filename)
    m=re.findall('exposure.*_(.*)_postisrccd[.]fits$',filename)
    
    return m[0]

In [23]:
file_tag_forsorting('exposure_2022031700310_postisrccd.fits')

'2022031700310'

In [24]:
#all_tags = [ file_tag_forsorting(filename)[0] for filename in all_selected_files]
all_tags = [ file_tag_forsorting(filename) for filename in all_selected_files]
all_tags=np.array(all_tags)
all_tags

array(['2022120700371', '2022120700436', '2022120700431', '2022120700417',
       '2022120700421', '2022120700426', '2022120700362', '2022120700459',
       '2022120700464', '2022120700402', '2022120700361', '2022120700376',
       '2022120700386', '2022120700454', '2022120700442', '2022120700367',
       '2022120700387', '2022120700381', '2022120700391', '2022120700377',
       '2022120700412', '2022120700401', '2022120700463', '2022120700447',
       '2022120700407', '2022120700366', '2022120700453', '2022120700372',
       '2022120700437', '2022120700422', '2022120700396', '2022120700443',
       '2022120700416', '2022120700411', '2022120700448', '2022120700432',
       '2022120700392', '2022120700357', '2022120700397', '2022120700382',
       '2022120700427', '2022120700458', '2022120700406'], dtype='<U13')

In [25]:
indexes=np.argsort(all_tags)

In [26]:
sorted_files=all_selected_files[indexes]

In [27]:
sorted_files

array(['exposure_2022120700357_postisrccd.fits',
       'exposure_2022120700361_postisrccd.fits',
       'exposure_2022120700362_postisrccd.fits',
       'exposure_2022120700366_postisrccd.fits',
       'exposure_2022120700367_postisrccd.fits',
       'exposure_2022120700371_postisrccd.fits',
       'exposure_2022120700372_postisrccd.fits',
       'exposure_2022120700376_postisrccd.fits',
       'exposure_2022120700377_postisrccd.fits',
       'exposure_2022120700381_postisrccd.fits',
       'exposure_2022120700382_postisrccd.fits',
       'exposure_2022120700386_postisrccd.fits',
       'exposure_2022120700387_postisrccd.fits',
       'exposure_2022120700391_postisrccd.fits',
       'exposure_2022120700392_postisrccd.fits',
       'exposure_2022120700396_postisrccd.fits',
       'exposure_2022120700397_postisrccd.fits',
       'exposure_2022120700401_postisrccd.fits',
       'exposure_2022120700402_postisrccd.fits',
       'exposure_2022120700406_postisrccd.fits',
       'exposure_202

In [28]:
N=len(sorted_files)

# Build the logbook from the headers

In [29]:
idx=0
#ccdir='/sps/lsst/groups/auxtel/data/2021/holo/FlipCleans'
ccdir=dir_images
all_dates=[]
all_files=[]
all_seqnum=[]
all_subdir=[]
all_indexes=[]
all_targets=[]
all_filters=[]
all_disperser=[]
all_airmass=[]
all_exposures=[]
all_focus=[]
all_seeing=[]
all_P=[]
all_T=[]
all_H=[]
all_W=[]
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header 
    all_indexes.append(idx)
    all_seqnum.append(hdr0['SEQNUM'])
    all_files.append(filename)
    all_subdir.append(ccdir)
    targetname=hdr0['OBJECT']
    all_targets.append(targetname)
    all_dates.append(hdr0['DATE-OBS'])
    all_airmass.append(hdr0['AMSTART'])
    all_filters.append(hdr0['FILTER'])
    all_disperser.append(hdr0['GRATING'])
    all_exposures.append(hdr0['EXPTIME'])
    all_focus.append(hdr0['FOCUSZ'])
    all_seeing.append(hdr0['SEEING'])
    all_P.append(hdr0['PRESSURE'])
    all_T.append(hdr0['AIRTEMP'])
    all_H.append(hdr0['HUMIDITY'])
    all_W.append(hdr0['WINDSPD'])
       
    
    idx+=1
    

# Make pandas dataframe

## Create pandas dataframe

In [30]:
df=pd.DataFrame()

In [31]:
len(all_airmass)

43

In [32]:
N=len(all_indexes)

## Fill with container info

In [33]:
df['index']=all_indexes
df['date']=all_dates
df['path']=all_subdir
df['file']=all_files
df['num']=all_seqnum
df['object']=all_targets
df['filter']=all_filters
df['disperser']=all_disperser
df['airmass']=all_airmass
df['exposure']=all_exposures
df['focus']=all_focus
df['seeing']=all_seeing
df['pressure']=all_P
df['temperature']=all_T
df['humidity']=all_H
df['windspd']=all_W

## Add dictionnary info

### Initialise new columns

In [34]:
Obj_posXpix=np.full(N,np.nan)
Obj_posYpix=np.full(N,np.nan)
Dx=np.full(N,np.nan)
Dy=np.full(N,np.nan)
run=np.full(N,np.nan)
quality=np.full(N,np.nan)
skip=np.full(N,"    ")

### Fill dictionnary info

In [35]:
# loop on rows index
for idx in np.arange(N):
    filename=all_files[idx]
    # check if filename is in dictionnary
    if filename in info.keys():
        (run_flag,quality_flag,xpos,ypos)=info[filename]
        Obj_posXpix[idx]=int(xpos)
        Obj_posYpix[idx]=int(ypos)
        run[idx]=run_flag
        if quality_flag==1:
            skip[idx] = "    "
            quality[idx]=1
        else:
            skip[idx] = "skip"
            quality[idx]=0

In [36]:
df['Obj-posXpix']=Obj_posXpix
df['Obj-posYpix']=Obj_posYpix
df['Dx']=Dx
df['Dy']=Dy
df['skip']=skip
df['run']=run
df['quality']=quality

In [37]:
pd.set_option('display.max_rows', None)

In [38]:
df

,index,date,path,file,num,object,filter,disperser,airmass,exposure,...,temperature,humidity,windspd,Obj-posXpix,Obj-posYpix,Dx,Dy,skip,run,quality
0,0,2022-12-08T04:21:39.848,/sps/lsst/groups/auxtel/data/2023/empty~holo4_...,exposure_2022120700357_postisrccd.fits,357,HD185975,empty_1,holo4_003,2.062400,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN
1,1,2022-12-08T04:24:19.011,/sps/lsst/groups/auxtel/data/2023/empty~holo4_...,exposure_2022120700361_postisrccd.fits,361,HD074000,empty_1,holo4_003,1.728743,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN
2,2,2022-12-08T04:25:01.980,/sps/lsst/groups/auxtel/data/2023/empty~holo4_...,exposure_2022120700362_postisrccd.fits,362,HD074000,empty_1,holo4_003,1.722192,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN
3,3,2022-12-08T04:27:21.436,/sps/lsst/groups/auxtel/data/2023/empty~holo4_...,exposure_2022120700366_postisrccd.fits,366,HD60753,empty_1,holo4_003,1.241016,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN
4,4,2022-12-08T04:28:04.366,/sps/lsst/groups/auxtel/data/2023/empty~holo4_...,exposure_2022120700367_postisrccd.fits,367,HD60753,empty_1,holo4_003,1.239308,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN
5,5,2022-12-08T04:30:21.301,/sps/lsst/groups/auxtel/data/2023/empty~holo4_...,exposure_2022120700371_postisrccd.fits,371,HD074000,empty_1,holo4_003,1.675438,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN
6,6,2022-12-08T04:31:04.229,/sps/lsst/groups/auxtel/data/2023/empty~holo4_...,exposure_2022120700372_postisrccd.fits,372,HD074000,empty_1,holo4_003,1.669413,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN
7,7,2022-12-08T04:43:49.617,/sps/lsst/groups/auxtel/data/2023/empty~holo4_...,exposure_2022120700376_postisrccd.fits,376,HD074000,empty_1,holo4_003,1.570405,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN
8,8,2022-12-08T04:44:32.479,/sps/lsst/groups/auxtel/data/2023/empty~holo4_...,exposure_2022120700377_postisrccd.fits,377,HD074000,empty_1,holo4_003,1.565322,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN
9,9,2022-12-08T04:46:52.811,/sps/lsst/groups/auxtel/data/2023/empty~holo4_...,exposure_2022120700381_postisrccd.fits,381,HD60753,empty_1,holo4_003,1.198119,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN


# Make logbook


In [39]:
N=len(df)
N

43

In [40]:
if N>0:
    df.to_excel(logbook_xlsx)
    df.to_csv(logbook_csv)

In [41]:
logbook_csv

'logbooks/auxtellogbook_empty~holo4_003_20221207_v6.0.csv'